In [4]:
import pandas as pd
from google.colab import files
import numpy as np

# Load the dataset
file_name = 'dataset.csv'
try:
    df = pd.read_csv(file_name)
    print(f"Successfully loaded {file_name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Number of rows: {len(df)}")
except Exception as e:
    print(f"Error loading {file_name}: {e}")
    file_name = input("Please enter the correct filename: ")
    df = pd.read_csv(file_name)

# Identify required columns and validate
required_columns = ['track_id', 'track_name', 'album_name', 'artists', 'explicit']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")

# Clean the data
df.dropna(subset=['track_id', 'track_name', 'album_name'], inplace=True)
df.drop_duplicates(subset=['track_id', 'track_name', 'album_name'], inplace=True)

# Dynamically handle track_genre if it exists
if 'track_genre' in df.columns:
    print("Processing track_genre column")
    # Check if genres are in a semicolon-separated format
    if df['track_genre'].str.contains(';').any():
        df['track_genre'] = df['track_genre'].str.split(';')
        genres_expanded = df[['track_genre']].explode('track_genre').drop_duplicates().reset_index(drop=True)
        genres_expanded['track_genre'] = genres_expanded['track_genre'].str.strip()
    else:
        genres_expanded = df[['track_genre']].drop_duplicates().reset_index(drop=True)

    genres_df = pd.DataFrame({
        'genre_name': genres_expanded['track_genre'].dropna().unique()
    })
else:
    print("No track_genre column found")
    # Create empty genres dataframe if track_genre doesn't exist
    genres_df = pd.DataFrame(columns=['genre_name'])

# Create Genres table with SERIAL ID (we'll use sequential IDs for now)
genres_df = genres_df.reset_index()
genres_df.columns = ['genre_id', 'genre_name']
genres_df['genre_id'] = genres_df['genre_id'] + 1  # Start from 1 instead of 0
genres_df.to_csv('genres.csv', index=False)

# Split artists and normalize the data
print("Processing artists column")
# Check if artists are in a semicolon-separated format
if df['artists'].str.contains(';').any():
    df['artists'] = df['artists'].str.split(';')
else:
    # Convert to list for consistent processing
    df['artists'] = df['artists'].apply(lambda x: [x] if pd.notna(x) else [])

artists_expanded = df[['artists']].explode('artists').drop_duplicates().reset_index(drop=True)
artists_expanded['artists'] = artists_expanded['artists'].str.strip()
artists_expanded = artists_expanded.rename(columns={'artists': 'artist_name'})
artists_expanded = artists_expanded.drop_duplicates().reset_index()
artists_expanded.columns = ['artist_id', 'artist_name']
artists_expanded['artist_id'] = artists_expanded['artist_id'] + 1  # Start from 1 instead of 0
artists_expanded.to_csv('artists.csv', index=False)

# Create a mapping dictionary for artist_name to artist_id for easier lookups
artist_id_map = dict(zip(artists_expanded['artist_name'], artists_expanded['artist_id']))

# Create Albums table with primary artist (first artist in the list)
print("Creating Albums table")
# We need to create a hashable version of the artists column for drop_duplicates
# First, create a copy to avoid modifying the original DataFrame
albums_temp = df[['album_name', 'artists']].copy()
# Convert lists to strings for deduplication purposes only
albums_temp['artists_str'] = albums_temp['artists'].apply(lambda x: str(x))
# Drop duplicates based on album_name and the string representation of artists
unique_albums = albums_temp.drop_duplicates(subset=['album_name', 'artists_str']).drop(columns=['artists_str'])
# Reset index for album_id generation
albums_df = unique_albums.reset_index(drop=True)

# Get the first artist for each album
albums_df['primary_artist'] = albums_df['artists'].apply(lambda x: x[0].strip() if isinstance(x, list) and len(x) > 0 else x)
# Map to artist_id
albums_df['artist_id'] = albums_df['primary_artist'].map(artist_id_map)
# Drop unnecessary columns and reset index to use as album_id
albums_df = albums_df.drop(columns=['artists', 'primary_artist']).reset_index()
albums_df.columns = ['album_id', 'album_name', 'artist_id']
albums_df['album_id'] = albums_df['album_id'] + 1  # Start from 1 instead of 0
albums_df.to_csv('albums.csv', index=False)

# Create a mapping dictionary for album_name to album_id
album_id_map = dict(zip(albums_df['album_name'], albums_df['album_id']))

# Create Tracks table - dynamically include all required columns and any optional ones that exist
print("Creating Tracks table")
track_cols = ['track_id', 'track_name', 'album_name']
optional_track_cols = ['popularity', 'duration_ms', 'explicit', 'track_genre']
track_cols.extend([col for col in optional_track_cols if col in df.columns])

tracks_df = df[track_cols].copy()

# Map album_name to album_id
tracks_df['album_id'] = tracks_df['album_name'].map(album_id_map)
tracks_df = tracks_df.drop(columns=['album_name'])

# Handle potential NaN values in optional columns
if 'popularity' in tracks_df.columns:
    tracks_df['popularity'] = tracks_df['popularity'].fillna(0)
if 'duration_ms' in tracks_df.columns:
    tracks_df['duration_ms'] = tracks_df['duration_ms'].fillna(0)
if 'explicit' in tracks_df.columns:
    # Handle explicit field - convert to boolean if it's not already
    if tracks_df['explicit'].dtype != bool:
        tracks_df['explicit'] = tracks_df['explicit'].apply(
            lambda x: True if pd.notna(x) and str(x).lower() in ['true', 't', 'yes', 'y', '1'] else False
        )

# Ensure all required columns exist
for col in ['track_id', 'track_name', 'album_id', 'explicit']:
    if col not in tracks_df.columns and col != 'explicit':  # explicit might be derived
        raise ValueError(f"Required column {col} missing from tracks table")
    elif col == 'explicit' and col not in tracks_df.columns:
        tracks_df['explicit'] = False  # Default value if missing

tracks_df.to_csv('tracks.csv', index=False)

# Create TrackGenres linking table (if track_genre exists)
if 'track_genre' in df.columns and not genres_df.empty:
    print("Creating TrackGenres linking table")
    # Create a mapping dictionary for genre_name to genre_id
    genre_id_map = dict(zip(genres_df['genre_name'], genres_df['genre_id']))

    # For tracks with list of genres
    if df['track_genre'].apply(lambda x: isinstance(x, list)).any():
        track_genres_df = df[['track_id', 'track_genre']].explode('track_genre').copy()
        track_genres_df['track_genre'] = track_genres_df['track_genre'].str.strip()
    else:
        track_genres_df = df[['track_id', 'track_genre']].copy()

    # Map genre_name to genre_id
    track_genres_df['genre_id'] = track_genres_df['track_genre'].map(genre_id_map)
    track_genres_df = track_genres_df.dropna(subset=['genre_id']).drop(columns=['track_genre'])
    track_genres_df = track_genres_df.drop_duplicates()
    track_genres_df.to_csv('track_genres.csv', index=False)

# Create Track-Artists Linking Table
print("Creating TrackArtists linking table")
track_artists_df = df[['track_id', 'artists']].explode('artists').copy()
track_artists_df['artists'] = track_artists_df['artists'].str.strip()
track_artists_df['artist_id'] = track_artists_df['artists'].map(artist_id_map)
track_artists_df = track_artists_df.drop(columns=['artists'])
track_artists_df = track_artists_df.dropna().drop_duplicates()
track_artists_df.to_csv('track_artists.csv', index=False)

# Dynamically create Audio Features table with available columns
print("Creating AudioFeatures table")
audio_feature_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
available_audio_cols = ['track_id'] + [col for col in audio_feature_cols if col in df.columns]

audio_features_df = df[available_audio_cols].copy()
audio_features_df = audio_features_df.dropna(subset=['track_id'])
audio_features_df = audio_features_df.drop_duplicates(subset=['track_id'])
audio_features_df.to_csv('audio_features.csv', index=False)

# Dynamically create Modes lookup table if mode column exists
if 'mode' in df.columns:
    print("Creating Modes lookup table")
    # Extract unique modes
    unique_modes = df['mode'].dropna().unique()

    # If modes are 0 and 1, use the standard Minor/Major mapping, otherwise use the values directly
    if set(unique_modes) == {0, 1} or set(unique_modes) == {0.0, 1.0}:
        modes_df = pd.DataFrame({
            'mode_id': [0, 1],
            'mode_name': ['Minor', 'Major']
        })
    else:
        modes_df = pd.DataFrame({
            'mode_id': unique_modes,
            'mode_name': [f"Mode {int(m)}" for m in unique_modes]
        })

    modes_df.to_csv('modes.csv', index=False)

# Dynamically create Keys lookup table if key column exists
if 'key' in df.columns:
    print("Creating Keys lookup table")
    # Extract unique keys
    unique_keys = sorted(df['key'].dropna().unique())

    # Standard key names for 0-11 if those are the values
    standard_keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

    if all(k in range(12) for k in unique_keys) or all(k in [float(i) for i in range(12)] for k in unique_keys):
        keys_df = pd.DataFrame({
            'key_id': unique_keys,
            'key_name': [standard_keys[int(k)] for k in unique_keys]
        })
    else:
        keys_df = pd.DataFrame({
            'key_id': unique_keys,
            'key_name': [f"Key {int(k)}" for k in unique_keys]
        })

    keys_df.to_csv('keys.csv', index=False)

# Dynamically create TimeSignatures lookup table if time_signature column exists
if 'time_signature' in df.columns:
    print("Creating TimeSignatures lookup table")
    time_signatures = sorted(df['time_signature'].dropna().unique())

    time_signatures_df = pd.DataFrame({
        'time_signature_id': time_signatures,
        'time_signature_value': time_signatures
    })
    time_signatures_df.to_csv('time_signatures.csv', index=False)
else:
    # Create default time signature table with 4/4
    time_signatures_df = pd.DataFrame({
        'time_signature_id': [4],
        'time_signature_value': [4]
    })
    time_signatures_df.to_csv('time_signatures.csv', index=False)

# Download the CSV files
print("Generating CSV files")
try:
    files.download('artists.csv')
    files.download('albums.csv')
    files.download('tracks.csv')
    files.download('audio_features.csv')
    files.download('track_artists.csv')
    if 'track_genre' in df.columns:
        files.download('genres.csv')
        files.download('track_genres.csv')
    if 'mode' in df.columns:
        files.download('modes.csv')
    if 'key' in df.columns:
        files.download('keys.csv')
    files.download('time_signatures.csv')
except Exception as e:
    print(f"Warning: Could not download files automatically: {e}")
    print("CSV files have been created and can be accessed from your current directory")

# Generate Create and Insert SQL Script
def save_insert_queries(dataframe, table_name, file, serial_columns=None):
    if serial_columns is None:
        serial_columns = []

    # Skip if dataframe is empty
    if dataframe.empty:
        return

    for _, row in dataframe.iterrows():
        # Filter out SERIAL columns for INSERT
        insert_columns = [col for col in dataframe.columns if col not in serial_columns]
        columns = ', '.join(insert_columns)

        values_list = []
        for col in insert_columns:
            x = row[col]
            if pd.isna(x):
                values_list.append("NULL")
            elif isinstance(x, str):
                escaped_x = x.replace("'", "''")
                values_list.append(f"'{escaped_x}'")
            else:
                values_list.append(str(x))

        values = ', '.join(values_list)

        # For tables with SERIAL columns, we need to specify the returning clause
        returning_clause = ""
        if serial_columns and table_name in ['Artists', 'Albums', 'Genres']:
            returning_clause = f" RETURNING {serial_columns[0]}"

        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values}){returning_clause};\n"
        file.write(insert_query)

print("Generating SQL script")
# Dynamically build the CREATE TABLE statements based on available columns
with open('create_and_insert_tables.sql', 'w') as f:
    # Artists table is always created
    f.write("""-- Create Artists Table
CREATE TABLE Artists (
    artist_id SERIAL PRIMARY KEY,
    artist_name VARCHAR(255) UNIQUE NOT NULL
);

-- Create Albums Table
CREATE TABLE Albums (
    album_id SERIAL PRIMARY KEY,
    album_name VARCHAR(255) NOT NULL,
    artist_id INT NOT NULL,
    FOREIGN KEY (artist_id) REFERENCES Artists(artist_id)
);
""")

    # Tracks table - dynamically build with available columns
    f.write("\n-- Create Tracks Table\n")
    f.write("CREATE TABLE Tracks (\n")
    f.write("    track_id VARCHAR(50) PRIMARY KEY,\n")
    f.write("    track_name VARCHAR(255) NOT NULL,\n")
    f.write("    album_id INT NOT NULL,\n")
    f.write("    explicit BOOLEAN NOT NULL,\n")

    # Optional track columns
    if 'popularity' in tracks_df.columns:
        f.write("    popularity INT,\n")
    if 'duration_ms' in tracks_df.columns:
        f.write("    duration_ms INT,\n")
    if 'track_genre' in tracks_df.columns:
        f.write("    track_genre VARCHAR(255),\n")

    f.write("    FOREIGN KEY (album_id) REFERENCES Albums(album_id)\n")
    f.write(");\n")

    # Genres table - only if track_genre exists
    if 'track_genre' in df.columns:
        f.write("""
-- Create Genres Table
CREATE TABLE Genres (
    genre_id SERIAL PRIMARY KEY,
    genre_name VARCHAR(255) UNIQUE NOT NULL
);

-- Create TrackGenres Table
CREATE TABLE TrackGenres (
    track_id VARCHAR(50) NOT NULL,
    genre_id INT NOT NULL,
    PRIMARY KEY (track_id, genre_id),
    FOREIGN KEY (track_id) REFERENCES Tracks(track_id),
    FOREIGN KEY (genre_id) REFERENCES Genres(genre_id)
);
""")

    # AudioFeatures table - dynamically build with available columns
    if len(available_audio_cols) > 1:  # More than just track_id
        f.write("\n-- Create AudioFeatures Table\n")
        f.write("CREATE TABLE AudioFeatures (\n")
        f.write("    track_id VARCHAR(50) PRIMARY KEY,\n")

        for col in available_audio_cols[1:]:  # Skip track_id
            if col == 'danceability' or col == 'energy' or col == 'speechiness' or col == 'acousticness' or col == 'instrumentalness' or col == 'liveness' or col == 'valence':
                f.write(f"    {col} FLOAT,\n")
            elif col == 'key' or col == 'mode' or col == 'time_signature':
                f.write(f"    {col} INT,\n")
            elif col == 'loudness' or col == 'tempo':
                f.write(f"    {col} FLOAT,\n")

        f.write("    FOREIGN KEY (track_id) REFERENCES Tracks(track_id)\n")
        f.write(");\n")

    # TrackArtists table is always created
    f.write("""
-- Create TrackArtists Table
CREATE TABLE TrackArtists (
    track_id VARCHAR(50) NOT NULL,
    artist_id INT NOT NULL,
    PRIMARY KEY (track_id, artist_id),
    FOREIGN KEY (track_id) REFERENCES Tracks(track_id),
    FOREIGN KEY (artist_id) REFERENCES Artists(artist_id)
);
""")

    # Mode table - if mode column exists
    if 'mode' in df.columns:
        f.write("""
-- Create Modes Table
CREATE TABLE Modes (
    mode_id INT PRIMARY KEY,
    mode_name VARCHAR(50) UNIQUE NOT NULL
);
""")

    # Keys table - if key column exists
    if 'key' in df.columns:
        f.write("""
-- Create Keys Table
CREATE TABLE Keys (
    key_id INT PRIMARY KEY,
    key_name VARCHAR(50) UNIQUE NOT NULL
);
""")

    # TimeSignatures table - always created
    f.write("""
-- Create TimeSignatures Table
CREATE TABLE TimeSignatures (
    time_signature_id INT PRIMARY KEY,
    time_signature_value INT UNIQUE NOT NULL
);
""")

    # Write Insert Data Statements
    f.write("\n\n-- Insert Data into Artists Table\n")
    save_insert_queries(artists_expanded, 'Artists', f, ['artist_id'])

    f.write("\n\n-- Insert Data into Albums Table\n")
    save_insert_queries(albums_df, 'Albums', f, ['album_id'])

    f.write("\n\n-- Insert Data into Tracks Table\n")
    save_insert_queries(tracks_df, 'Tracks', f)

    if 'track_genre' in df.columns and not genres_df.empty:
        f.write("\n\n-- Insert Data into Genres Table\n")
        save_insert_queries(genres_df, 'Genres', f, ['genre_id'])

        if 'track_genres_df' in locals() and not track_genres_df.empty:
            f.write("\n\n-- Insert Data into TrackGenres Table\n")
            save_insert_queries(track_genres_df, 'TrackGenres', f)

    if len(available_audio_cols) > 1:  # More than just track_id
        f.write("\n\n-- Insert Data into AudioFeatures Table\n")
        save_insert_queries(audio_features_df, 'AudioFeatures', f)

    f.write("\n\n-- Insert Data into TrackArtists Table\n")
    save_insert_queries(track_artists_df, 'TrackArtists', f)

    if 'mode' in df.columns and 'modes_df' in locals() and not modes_df.empty:
        f.write("\n\n-- Insert Data into Modes Table\n")
        save_insert_queries(modes_df, 'Modes', f)

    if 'key' in df.columns and 'keys_df' in locals() and not keys_df.empty:
        f.write("\n\n-- Insert Data into Keys Table\n")
        save_insert_queries(keys_df, 'Keys', f)

    f.write("\n\n-- Insert Data into TimeSignatures Table\n")
    save_insert_queries(time_signatures_df, 'TimeSignatures', f)

# Download SQL file
try:
    files.download('create_and_insert_tables.sql')
    print("SQL script successfully generated and downloaded")
except Exception as e:
    print(f"Warning: Could not download SQL file automatically: {e}")
    print("SQL file has been created and can be accessed from your current directory")

print("Processing complete!")


# Dynamically create Audio Features table with available columns
print("Creating AudioFeatures table")
audio_feature_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
available_audio_cols = ['track_id'] + [col for col in audio_feature_cols if col in df.columns]

audio_features_df = df[available_audio_cols].copy()
audio_features_df = audio_features_df.dropna(subset=['track_id'])
audio_features_df = audio_features_df.drop_duplicates(subset=['track_id'])
audio_features_df.to_csv('audio_features.csv', index=False)

# Dynamically create Modes lookup table if mode column exists
if 'mode' in df.columns:
    print("Creating Modes lookup table")
    # Extract unique modes
    unique_modes = df['mode'].dropna().unique()

    # If modes are 0 and 1, use the standard Minor/Major mapping, otherwise use the values directly
    if set(unique_modes) == {0, 1} or set(unique_modes) == {0.0, 1.0}:
        modes_df = pd.DataFrame({
            'mode_id': [0, 1],
            'mode_name': ['Minor', 'Major']
        })
    else:
        modes_df = pd.DataFrame({
            'mode_id': unique_modes,
            'mode_name': [f"Mode {int(m)}" for m in unique_modes]
        })

    modes_df.to_csv('modes.csv', index=False)

# Dynamically create Keys lookup table if key column exists
if 'key' in df.columns:
    print("Creating Keys lookup table")
    # Extract unique keys
    unique_keys = sorted(df['key'].dropna().unique())

    # Standard key names for 0-11 if those are the values
    standard_keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

    if all(k in range(12) for k in unique_keys) or all(k in [float(i) for i in range(12)] for k in unique_keys):
        keys_df = pd.DataFrame({
            'key_id': unique_keys,
            'key_name': [standard_keys[int(k)] for k in unique_keys]
        })
    else:
        keys_df = pd.DataFrame({
            'key_id': unique_keys,
            'key_name': [f"Key {int(k)}" for k in unique_keys]
        })

    keys_df.to_csv('keys.csv', index=False)

# Dynamically create TimeSignatures lookup table if time_signature column exists
if 'time_signature' in df.columns:
    print("Creating TimeSignatures lookup table")
    time_signatures = sorted(df['time_signature'].dropna().unique())

    time_signatures_df = pd.DataFrame({
        'time_signature_id': time_signatures,
        'time_signature_value': time_signatures
    })
    time_signatures_df.to_csv('time_signatures.csv', index=False)
else:
    # Create default time signature table with 4/4
    time_signatures_df = pd.DataFrame({
        'time_signature_id': [4],
        'time_signature_value': [4]
    })
    time_signatures_df.to_csv('time_signatures.csv', index=False)

# Download the CSV files
print("Generating CSV files")
try:
    files.download('artists.csv')
    files.download('albums.csv')
    files.download('tracks.csv')
    files.download('audio_features.csv')
    files.download('track_artists.csv')
    if 'track_genre' in df.columns:
        files.download('genres.csv')
        files.download('track_genres.csv')
    if 'mode' in df.columns:
        files.download('modes.csv')
    if 'key' in df.columns:
        files.download('keys.csv')
    files.download('time_signatures.csv')
except Exception as e:
    print(f"Warning: Could not download files automatically: {e}")
    print("CSV files have been created and can be accessed from your current directory")

# Generate Create and Insert SQL Script
def save_insert_queries(dataframe, table_name, file, serial_columns=None):
    if serial_columns is None:
        serial_columns = []

    for _, row in dataframe.iterrows():
        # Filter out SERIAL columns for INSERT
        insert_columns = [col for col in dataframe.columns if col not in serial_columns]
        columns = ', '.join(insert_columns)

        values_list = []
        for col in insert_columns:
            x = row[col]
            if pd.isna(x):
                values_list.append("NULL")
            elif isinstance(x, str):
                escaped_x = x.replace("'", "''")
                values_list.append(f"'{escaped_x}'")
            else:
                values_list.append(str(x))

        values = ', '.join(values_list)

        # For tables with SERIAL columns, we need to specify the returning clause
        returning_clause = ""
        if serial_columns and table_name in ['Artists', 'Albums', 'Genres']:
            returning_clause = f" RETURNING {serial_columns[0]}"

        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values}){returning_clause};\n"
        file.write(insert_query)

# Save the insert queries for each table to the same SQL file
with open('create_and_insert_tables.sql', 'w') as f:
    # Write Create Table Scripts for the new schema
    f.write("""-- Create Artists Table
CREATE TABLE Artists (
    artist_id SERIAL PRIMARY KEY,
    artist_name VARCHAR(255) UNIQUE NOT NULL
);

-- Create Albums Table
CREATE TABLE Albums (
    album_id SERIAL PRIMARY KEY,
    album_name VARCHAR(255) NOT NULL,
    artist_id INT NOT NULL,
    FOREIGN KEY (artist_id) REFERENCES Artists(artist_id)
);

-- Create Tracks Table
CREATE TABLE Tracks (
    track_id VARCHAR(50) PRIMARY KEY,
    track_name VARCHAR(255) NOT NULL,
    album_id INT NOT NULL,
    explicit BOOLEAN NOT NULL,
    popularity INT,
    duration_ms INT,
    track_genre VARCHAR(255),
    FOREIGN KEY (album_id) REFERENCES Albums(album_id)
);

-- Create Genres Table
CREATE TABLE Genres (
    genre_id SERIAL PRIMARY KEY,
    genre_name VARCHAR(255) UNIQUE NOT NULL
);

-- Create TrackGenres Table
CREATE TABLE TrackGenres (
    track_id VARCHAR(50) NOT NULL,
    genre_id INT NOT NULL,
    PRIMARY KEY (track_id, genre_id),
    FOREIGN KEY (track_id) REFERENCES Tracks(track_id),
    FOREIGN KEY (genre_id) REFERENCES Genres(genre_id)
);

-- Create AudioFeatures Table
CREATE TABLE AudioFeatures (
    track_id VARCHAR(50) PRIMARY KEY,
    danceability FLOAT,
    energy FLOAT,
    key INT,
    loudness FLOAT,
    mode INT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INT,
    FOREIGN KEY (track_id) REFERENCES Tracks(track_id)
);

-- Create TrackArtists Table
CREATE TABLE TrackArtists (
    track_id VARCHAR(50) NOT NULL,
    artist_id INT NOT NULL,
    PRIMARY KEY (track_id, artist_id),
    FOREIGN KEY (track_id) REFERENCES Tracks(track_id),
    FOREIGN KEY (artist_id) REFERENCES Artists(artist_id)
);

-- Create Modes Table
CREATE TABLE Modes (
    mode_id INT PRIMARY KEY,
    mode_name VARCHAR(50) UNIQUE NOT NULL
);

-- Create Keys Table
CREATE TABLE Keys (
    key_id INT PRIMARY KEY,
    key_name VARCHAR(50) UNIQUE NOT NULL
);

-- Create TimeSignatures Table
CREATE TABLE TimeSignatures (
    time_signature_id INT PRIMARY KEY,
    time_signature_value INT UNIQUE NOT NULL
);
""")

    # Write Insert Data Statements
    f.write("\n\n-- Insert Data into Artists Table\n")
    save_insert_queries(artists_expanded, 'Artists', f, ['artist_id'])

    f.write("\n\n-- Insert Data into Albums Table\n")
    save_insert_queries(albums_df, 'Albums', f, ['album_id'])

    f.write("\n\n-- Insert Data into Tracks Table\n")
    save_insert_queries(tracks_df, 'Tracks', f)

    if 'track_genre' in df.columns and not genres_df.empty:
        f.write("\n\n-- Insert Data into Genres Table\n")
        save_insert_queries(genres_df, 'Genres', f, ['genre_id'])

        f.write("\n\n-- Insert Data into TrackGenres Table\n")
        save_insert_queries(track_genres_df, 'TrackGenres', f)

    f.write("\n\n-- Insert Data into AudioFeatures Table\n")
    save_insert_queries(audio_features_df, 'AudioFeatures', f)

    f.write("\n\n-- Insert Data into TrackArtists Table\n")
    save_insert_queries(track_artists_df, 'TrackArtists', f)

    f.write("\n\n-- Insert Data into Modes Table\n")
    save_insert_queries(modes_df, 'Modes', f)

    f.write("\n\n-- Insert Data into Keys Table\n")
    save_insert_queries(keys_df, 'Keys', f)

    f.write("\n\n-- Insert Data into TimeSignatures Table\n")
    save_insert_queries(time_signatures_df, 'TimeSignatures', f)

# Download SQL file
files.download('create_and_insert_tables.sql')

<ipython-input-4-3a514495751e>:8: DtypeWarning: Columns (1,2,3,4,7,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Successfully loaded dataset.csv
Columns: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']
Number of rows: 113993
Processing track_genre column
Processing artists column
Creating Albums table
Creating Tracks table
Creating TrackGenres linking table
Creating TrackArtists linking table
Creating AudioFeatures table
Creating Modes lookup table
Creating Keys lookup table
Creating TimeSignatures lookup table
Generating CSV files


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>